# STS Benchmark Datasets

In [1]:
%load_ext autotime
import numpy as np
import logging
import pandas as pd
import pathlib

from gensim.models import Word2Vec, FastText
from gensim.models.keyedvectors import KeyedVectors, FastTextKeyedVectors

from fse.models.average import Average
from fse.models.sif import SIF
from fse.models.usif import uSIF
from fse.models.inputs import IndexedSentence

from re import sub
from scipy.stats import pearsonr

import re
from nltk import word_tokenize

logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
                    level=logging.INFO
                   )

# This part is directly taken form the uSIF implementation
not_punc = re.compile('.*[A-Za-z0-9].*')

def preprocess(t):
    t = t.lower().strip("';.:()").strip('"')
    t = 'not' if t == "n't" else t
    return re.split(r'[-]', t)

def normalize_text(sentence):
    tokens = []
    
    for token in word_tokenize(sentence):
        if not_punc.match(token):
            tokens = tokens + preprocess(token)
    return tokens

def compute_similarities(task_length, model):
    sims = []
    for i, j in zip(range(task_length), range(task_length, 2*task_length)):
        sims.append(model.sv.similarity(i,j))
    return sims

file= "data/stsbenchmark/sts-test.csv"
similarities, sent_a, sent_b = [], [], []
with open(file, "r") as f:
    for l in f:
        line = l.rstrip().split("\t")
        similarities.append(float(line[4]))
        sent_a.append(normalize_text(line[5]))
        sent_b.append(normalize_text(line[6]))
similarities = np.array(similarities)
assert len(similarities) == len(sent_a) == len(sent_b)
task_length = len(similarities)
final_sents = sent_a + sent_b
sents = [IndexedSentence(s, i) for i,s in enumerate(final_sents)]

In [2]:
models, results = {}, {}

time: 625 µs


In [3]:
avg_glove = Average.load("data/vectors/glove.model")
models[f"CBOW-Glove"] = Average(avg_glove.wv, wv_mapfile_path="data/vectors/glove")
models[f"SIF-Glove"] = SIF(avg_glove.wv, components=15)
models[f"uSIF-Glove"] = uSIF(avg_glove.wv,length=11)

2019-08-20 16:32:52,869 : MainThread : INFO : loading Average object from data/vectors/glove.model
2019-08-20 16:32:57,377 : MainThread : INFO : loading wv recursively from data/vectors/glove.model.wv.* with mmap=None
2019-08-20 16:32:57,377 : MainThread : INFO : loading sv recursively from data/vectors/glove.model.sv.* with mmap=None
2019-08-20 16:32:57,378 : MainThread : INFO : loading prep recursively from data/vectors/glove.model.prep.* with mmap=None
2019-08-20 16:32:57,378 : MainThread : INFO : loaded data/vectors/glove.model
/Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/models/base_s2v.py:157: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  "C extension not loaded, training/inferring will be slow. "
2019-08-20 16:32:57,395 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast

time: 4.56 s


In [4]:
avg_w2v = Average.load("data/vectors/w2v.model")
models[f"CBOW-W2V"] = Average(avg_w2v.wv, wv_mapfile_path="data/vectors/w2v")
models[f"SIF-W2V"] = SIF(avg_w2v.wv, components=10)
models[f"uSIF-W2V"] = uSIF(avg_w2v.wv, length=11)

2019-08-20 16:32:57,439 : MainThread : INFO : loading Average object from data/vectors/w2v.model
2019-08-20 16:33:03,187 : MainThread : INFO : loading wv recursively from data/vectors/w2v.model.wv.* with mmap=None
2019-08-20 16:33:03,188 : MainThread : INFO : loading sv recursively from data/vectors/w2v.model.sv.* with mmap=None
2019-08-20 16:33:03,189 : MainThread : INFO : loading prep recursively from data/vectors/w2v.model.prep.* with mmap=None
2019-08-20 16:33:03,189 : MainThread : INFO : loaded data/vectors/w2v.model
/Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/models/base_s2v.py:157: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  "C extension not loaded, training/inferring will be slow. "
2019-08-20 16:33:03,208 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_

time: 5.8 s


In [5]:
avg_ft = Average.load("data/vectors/ft.model")
models[f"CBOW-FT"] = Average(avg_ft.wv, wv_mapfile_path="data/vectors/ft")
models[f"SIF-FT"] = SIF(avg_ft.wv, components=10)
models[f"uSIF-FT"] = uSIF(avg_ft.wv, length=11)

2019-08-20 16:33:03,247 : MainThread : INFO : loading Average object from data/vectors/ft.model
2019-08-20 16:33:08,000 : MainThread : INFO : loading wv recursively from data/vectors/ft.model.wv.* with mmap=None
2019-08-20 16:33:08,001 : MainThread : INFO : loading sv recursively from data/vectors/ft.model.sv.* with mmap=None
2019-08-20 16:33:08,002 : MainThread : INFO : loading prep recursively from data/vectors/ft.model.prep.* with mmap=None
2019-08-20 16:33:08,002 : MainThread : INFO : loaded data/vectors/ft.model
/Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/models/base_s2v.py:157: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  "C extension not loaded, training/inferring will be slow. "
2019-08-20 16:33:08,020 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embed

time: 4.83 s


In [6]:
paranmt = KeyedVectors.load("/Users/oliverborchers/Desktop/GSDEV/Models/Static/paranmt.model")
models[f"CBOW-Paranmt"] = Average(paranmt, lang_freq="en")
models[f"SIF-Paranmt"] = SIF(paranmt, components=10, lang_freq="en")
models[f"uSIF-Paranmt"] = uSIF(paranmt, length=11, lang_freq="en")

2019-08-20 16:33:08,083 : MainThread : INFO : loading Word2VecKeyedVectors object from /Users/oliverborchers/Desktop/GSDEV/Models/Static/paranmt.model
2019-08-20 16:33:08,307 : MainThread : INFO : loading vectors from /Users/oliverborchers/Desktop/GSDEV/Models/Static/paranmt.model.vectors.npy with mmap=None
2019-08-20 16:33:08,366 : MainThread : INFO : setting ignored attribute vectors_norm to None
2019-08-20 16:33:08,367 : MainThread : INFO : loaded /Users/oliverborchers/Desktop/GSDEV/Models/Static/paranmt.model
/Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/models/base_s2v.py:157: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  "C extension not loaded, training/inferring will be slow. "
2019-08-20 16:33:08,369 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en
2019-08-20 16:33:08,564 : MainThread : INFO : no fre

time: 634 ms


In [7]:
for k, m in models.items():
    m_type  = k.split("-")[0]
    emb_type = k.split("-")[1]
    m.train(sents)
    r = pearsonr(similarities, compute_similarities(task_length, m))[0].round(4) * 100
    results[f"{m_type}-{emb_type}"] = r
    print(k, f"{r:2.2f}")

2019-08-20 16:33:08,724 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-20 16:33:08,734 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2019-08-20 16:33:09,498 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 2195875 vocabulary: 2524 MB (2 GB)
2019-08-20 16:33:09,498 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2019-08-20 16:33:09,542 : MainThread : INFO : begin training
2019-08-20 16:33:09,660 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-20 16:33:09,661 : MainThread : INFO : training on 2758 effective sentences with 27351 effective words took 0s with 23275 sentences/s
2019-08-20 16:33:09,687 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-20 16:33:09,695 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words


CBOW-Glove 40.41


2019-08-20 16:33:10,461 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 2195875 vocabulary: 2524 MB (2 GB)
2019-08-20 16:33:10,462 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2019-08-20 16:33:10,471 : MainThread : INFO : pre-computing SIF weights for 2195875 words
2019-08-20 16:33:12,827 : MainThread : INFO : begin training
2019-08-20 16:33:12,932 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-20 16:33:12,933 : MainThread : INFO : computing 15 principal components
2019-08-20 16:33:12,975 : MainThread : INFO : removing 15 principal components
2019-08-20 16:33:12,977 : MainThread : INFO : training on 2758 effective sentences with 27351 effective words took 0s with 25962 sentences/s
2019-08-20 16:33:13,004 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-20 16:33:13,012 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and

SIF-Glove 71.95


2019-08-20 16:33:13,784 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 2195875 vocabulary: 2524 MB (2 GB)
2019-08-20 16:33:13,784 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2019-08-20 16:33:13,791 : MainThread : INFO : pre-computing uSIF weights for 2195875 words
2019-08-20 16:33:23,183 : MainThread : INFO : begin training
2019-08-20 16:33:23,311 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-20 16:33:23,312 : MainThread : INFO : computing 5 principal components
2019-08-20 16:33:23,351 : MainThread : INFO : training on 2758 effective sentences with 27351 effective words took 0s with 21480 sentences/s
2019-08-20 16:33:23,394 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-20 16:33:23,407 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words


...

uSIF-FT 69.89
CBOW-Paranmt 79.85


2019-08-20 16:33:58,252 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2019-08-20 16:33:58,297 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 77224 vocabulary: 91 MB (0 GB)
2019-08-20 16:33:58,299 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2019-08-20 16:33:58,307 : MainThread : INFO : pre-computing SIF weights for 77224 words
2019-08-20 16:33:58,397 : MainThread : INFO : begin training
2019-08-20 16:33:58,500 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-20 16:33:58,501 : MainThread : INFO : computing 10 principal components
2019-08-20 16:33:58,520 : MainThread : INFO : removing 10 principal components
2019-08-20 16:33:58,522 : MainThread : INFO : training on 2758 effective sentences with 27441 effective words took 0s with 26568 sentences/s
2019-08-20 16:33:58,553 : MainThread : INFO : scanning all indexed sentences and their

SIF-Paranmt 76.73


2019-08-20 16:33:58,914 : MainThread : INFO : begin training
2019-08-20 16:33:59,016 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-20 16:33:59,017 : MainThread : INFO : computing 5 principal components
2019-08-20 16:33:59,043 : MainThread : INFO : training on 2758 effective sentences with 27441 effective words took 0s with 26684 sentences/s


uSIF-Paranmt 79.20
time: 50.4 s


In [8]:
pd.DataFrame.from_dict(results, orient="index", columns=["Pearson"])

,Pearson
CBOW-Glove,40.41
SIF-Glove,71.95
uSIF-Glove,68.28
CBOW-W2V,61.54
SIF-W2V,71.12
uSIF-W2V,67.46
CBOW-FT,48.53
SIF-FT,73.42
uSIF-FT,69.89
CBOW-Paranmt,79.85


time: 20.7 ms


# Additionally: Write glove vectors to KeyedVectors

# Read Paranmt Vectors from Disk